In [1]:
# ! pip install gymnasium

In [2]:
#基础逻辑理解清楚了，现在得notebook把代码来跑通来
import argparse
import logging
import multiprocessing as mp
import os
import pytz
import shutil
from datetime import datetime

In [3]:
parser = argparse.ArgumentParser(description='LEGO-Prover')
parser.add_argument('--resume', action='store_true',
                    help='whether to resume from the checkpoint')
parser.add_argument('--data_split', type=str, choices=['valid', 'test'], 
                    default='valid', help='data split to use in the miniF2F dataset')
parser.add_argument('--ckpt_dir', type=str, default='checkpoints/lego_prover_valid_2023_10_27',
                    help='path to the checkpoint directory')
parser.add_argument('--isabelle_path', type=str, default='/data2/wanghaiming/Isabelle2022/',
                    help='path to the Isabelle2022 directory')
parser.add_argument('--model_name', type=str, choices=["gpt-3.5-turbo", "gpt-4"], 
                    default='gpt-3.5-turbo', help='OpenAI model name')
parser.add_argument('--temperature', type=float, default=0.7,
                    help='temperature for sampling the LLM')
parser.add_argument('--num_prover', type=int, default=3,
                    help='number of prover processes')
parser.add_argument('--num_evolver', type=int, default=8,
                    help='number of evolver processes')
parser.add_argument('--num_attempts', type=int, default=100,
                    help='number of proving attempts for each problem in the dataset')
args = parser.parse_args([])
# for arg in args:
#     print(arg)


In [4]:
resume = args.resume
data_split = args.data_split
ckpt_dir = args.ckpt_dir
isabelle_path = args.isabelle_path
model_name = args.model_name
temperature = args.temperature
number_of_prover_processes = args.num_prover
number_of_evolver_processes = args.num_evolver
number_of_prover_attempts = args.num_attempts

if os.path.exists(ckpt_dir) and not resume:
    text = input(f"the checkpoint directory {ckpt_dir} is already exist, and" + \
                 f"you are not resuming from it, do you want to delete it? (y/n)")
    if "y" in text.lower():
        shutil.rmtree(ckpt_dir, ignore_errors=True)
        resume = False
    else:
        resume = True


the checkpoint directory checkpoints/lego_prover_valid_2023_10_27 is already exist, andyou are not resuming from it, do you want to delete it? (y/n) y


In [5]:
## file utils
"""
File system utils.
"""
import collections
import os
import pickle
import sys
import errno
import shutil
import glob

# import pwd
import codecs
import hashlib
import tarfile
import fnmatch
import tempfile
from datetime import datetime
from socket import gethostname
import logging


f_ext = os.path.splitext

f_size = os.path.getsize

is_file = os.path.isfile

is_dir = os.path.isdir

get_dir = os.path.dirname


def host_name():
    "Get host name, alias with ``socket.gethostname()``"
    return gethostname()


def host_id():
    """
    Returns: first part of hostname up to '.'
    """
    return host_name().split(".")[0]


def utf_open(fname, mode):
    """
    Wrapper for codecs.open
    """
    return codecs.open(fname, mode=mode, encoding="utf-8")


def is_sequence(obj):
    """
    Returns:
      True if the sequence is a collections.Sequence and not a string.
    """
    return isinstance(obj, collections.abc.Sequence) and not isinstance(obj, str)


def pack_varargs(args):
    """
    Pack *args or a single list arg as list

    def f(*args):
        arg_list = pack_varargs(args)
        # arg_list is now packed as a list
    """
    assert isinstance(args, tuple), "please input the tuple `args` as in *args"
    if len(args) == 1 and is_sequence(args[0]):
        return args[0]
    else:
        return args


def f_not_empty(*fpaths):
    """
    Returns:
        True if and only if the file exists and file size > 0
          if fpath is a dir, if and only if dir exists and has at least 1 file
    """
    fpath = f_join(*fpaths)
    if not os.path.exists(fpath):
        return False

    if os.path.isdir(fpath):
        return len(os.listdir(fpath)) > 0
    else:
        return os.path.getsize(fpath) > 0


def f_expand(fpath):
    return os.path.expandvars(os.path.expanduser(fpath))


def f_exists(*fpaths):
    return os.path.exists(f_join(*fpaths))


def f_join(*fpaths):
    """
    join file paths and expand special symbols like `~` for home dir
    """
    fpaths = pack_varargs(fpaths)
    fpath = f_expand(os.path.join(*fpaths))
    if isinstance(fpath, str):
        fpath = fpath.strip()
    return fpath


def f_listdir(
    *fpaths,
    filter_ext=None,
    filter=None,
    sort=True,
    full_path=False,
    nonexist_ok=True,
    recursive=False,
):
    """
    Args:
        full_path: True to return full paths to the dir contents
        filter: function that takes in file name and returns True to include
        nonexist_ok: True to return [] if the dir is non-existent, False to raise
        sort: sort the file names by alphabetical
        recursive: True to use os.walk to recursively list files. Note that `filter`
            will be applied to the relative path string to the root dir.
            e.g. filter will take "a/data1.txt" and "a/b/data3.txt" as input, instead of
            just the base file names "data1.txt" and "data3.txt".
            if False, will simply call os.listdir()
    """
    assert not (filter_ext and filter), "filter_ext and filter are mutually exclusive"
    dir_path = f_join(*fpaths)
    if not os.path.exists(dir_path) and nonexist_ok:
        return []
    if recursive:
        files = [
            os.path.join(os.path.relpath(root, dir_path), file)
            for root, _, files in os.walk(dir_path)
            for file in files
        ]
    else:
        files = os.listdir(dir_path)
    if filter is not None:
        files = [f for f in files if filter(f)]
    elif filter_ext is not None:
        files = [f for f in files if f.endswith(filter_ext)]
    if sort:
        files.sort()
    if full_path:
        return [os.path.join(dir_path, f) for f in files]
    else:
        return files


def f_mkdir(*fpaths):
    """
    Recursively creates all the subdirs
    If exist, do nothing.
    """
    fpath = f_join(*fpaths)
    os.makedirs(fpath, exist_ok=True)
    return fpath


def f_mkdir_in_path(*fpaths):
    """
    fpath is a file,
    recursively creates all the parent dirs that lead to the file
    If exist, do nothing.
    """
    os.makedirs(get_dir(f_join(*fpaths)), exist_ok=True)


def last_part_in_path(fpath):
    """
    https://stackoverflow.com/questions/3925096/how-to-get-only-the-last-part-of-a-path-in-python
    """
    return os.path.basename(os.path.normpath(f_expand(fpath)))


def is_abs_path(*fpath):
    return os.path.isabs(f_join(*fpath))


def is_relative_path(*fpath):
    return not is_abs_path(f_join(*fpath))


def f_time(*fpath):
    "File modification time"
    return str(os.path.getctime(f_join(*fpath)))


def f_append_before_ext(fpath, suffix):
    """
    Append a suffix to file name and retain its extension
    """
    name, ext = f_ext(fpath)
    return name + suffix + ext


def f_add_ext(fpath, ext):
    """
    Append an extension if not already there
    Args:
      ext: will add a preceding `.` if doesn't exist
    """
    if not ext.startswith("."):
        ext = "." + ext
    if fpath.endswith(ext):
        return fpath
    else:
        return fpath + ext


def f_has_ext(fpath, ext):
    "Test if file path is a text file"
    _, actual_ext = f_ext(fpath)
    return actual_ext == "." + ext.lstrip(".")


def f_glob(*fpath):
    return glob.glob(f_join(*fpath), recursive=True)


def f_remove(*fpath, verbose=False, dry_run=False):
    """
    If exist, remove. Supports both dir and file. Supports glob wildcard.
    """
    assert isinstance(verbose, bool)
    fpath = f_join(fpath)
    if dry_run:
        print("Dry run, delete:", fpath)
        return
    for f in glob.glob(fpath):
        try:
            shutil.rmtree(f)
        except OSError as e:
            if e.errno == errno.ENOTDIR:
                try:
                    os.remove(f)
                except:  # final resort safeguard
                    pass
    if verbose:
        print(f'Deleted "{fpath}"')


def f_copy(fsrc, fdst, ignore=None, include=None, exists_ok=True, verbose=False):
    """
    Supports both dir and file. Supports glob wildcard.
    """
    fsrc, fdst = f_expand(fsrc), f_expand(fdst)
    for f in glob.glob(fsrc):
        try:
            f_copytree(f, fdst, ignore=ignore, include=include, exist_ok=exists_ok)
        except OSError as e:
            if e.errno == errno.ENOTDIR:
                shutil.copy(f, fdst)
            else:
                raise
    if verbose:
        print(f'Copied "{fsrc}" to "{fdst}"')


def _f_copytree(
    src,
    dst,
    symlinks=False,
    ignore=None,
    exist_ok=True,
    copy_function=shutil.copy2,
    ignore_dangling_symlinks=False,
):
    """Copied from python standard lib shutil.copytree
    except that we allow exist_ok
    Use f_copytree as entry
    """
    names = os.listdir(src)
    if ignore is not None:
        ignored_names = ignore(src, names)
    else:
        ignored_names = set()

    os.makedirs(dst, exist_ok=exist_ok)
    errors = []
    for name in names:
        if name in ignored_names:
            continue
        srcname = os.path.join(src, name)
        dstname = os.path.join(dst, name)
        try:
            if os.path.islink(srcname):
                linkto = os.readlink(srcname)
                if symlinks:
                    # We can't just leave it to `copy_function` because legacy
                    # code with a custom `copy_function` may rely on copytree
                    # doing the right thing.
                    os.symlink(linkto, dstname)
                    shutil.copystat(srcname, dstname, follow_symlinks=not symlinks)
                else:
                    # ignore dangling symlink if the flag is on
                    if not os.path.exists(linkto) and ignore_dangling_symlinks:
                        continue
                    # otherwise let the copy occurs. copy2 will raise an error
                    if os.path.isdir(srcname):
                        _f_copytree(
                            srcname, dstname, symlinks, ignore, exist_ok, copy_function
                        )
                    else:
                        copy_function(srcname, dstname)
            elif os.path.isdir(srcname):
                _f_copytree(srcname, dstname, symlinks, ignore, exist_ok, copy_function)
            else:
                # Will raise a SpecialFileError for unsupported file types
                copy_function(srcname, dstname)
        # catch the Error from the recursive copytree so that we can
        # continue with other files
        except shutil.Error as err:
            errors.extend(err.args[0])
        except OSError as why:
            errors.append((srcname, dstname, str(why)))
    try:
        shutil.copystat(src, dst)
    except OSError as why:
        # Copying file access times may fail on Windows
        if getattr(why, "winerror", None) is None:
            errors.append((src, dst, str(why)))
    if errors:
        raise shutil.Error(errors)
    return dst


def _include_patterns(*patterns):
    """Factory function that can be used with copytree() ignore parameter.

    Arguments define a sequence of glob-style patterns
    that are used to specify what files to NOT ignore.
    Creates and returns a function that determines this for each directory
    in the file hierarchy rooted at the source directory when used with
    shutil.copytree().
    """

    def _ignore_patterns(path, names):
        keep = set(
            name for pattern in patterns for name in fnmatch.filter(names, pattern)
        )
        ignore = set(
            name
            for name in names
            if name not in keep and not os.path.isdir(os.path.join(path, name))
        )
        return ignore

    return _ignore_patterns


def f_copytree(fsrc, fdst, symlinks=False, ignore=None, include=None, exist_ok=True):
    fsrc, fdst = f_expand(fsrc), f_expand(fdst)
    assert (ignore is None) or (
        include is None
    ), "ignore= and include= are mutually exclusive"
    if ignore:
        ignore = shutil.ignore_patterns(*ignore)
    elif include:
        ignore = _include_patterns(*include)
    _f_copytree(fsrc, fdst, ignore=ignore, symlinks=symlinks, exist_ok=exist_ok)


def f_move(fsrc, fdst):
    fsrc, fdst = f_expand(fsrc), f_expand(fdst)
    for f in glob.glob(fsrc):
        shutil.move(f, fdst)


def f_split_path(fpath, normpath=True):
    """
    Splits path into a list of its component folders

    Args:
        normpath: call os.path.normpath to remove redundant '/' and
            up-level references like ".."
    """
    if normpath:
        fpath = os.path.normpath(fpath)
    allparts = []
    while 1:
        parts = os.path.split(fpath)
        if parts[0] == fpath:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == fpath:  # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            fpath = parts[0]
            allparts.insert(0, parts[1])
    return allparts


def get_script_dir():
    """
    Returns: the dir of current script
    """
    return os.path.dirname(os.path.realpath(sys.argv[0]))


def get_script_file_name():
    """
    Returns: the dir of current script
    """
    return os.path.basename(sys.argv[0])


def get_script_self_path():
    """
    Returns: the dir of current script
    """
    return os.path.realpath(sys.argv[0])


def get_parent_dir(location, abspath=False):
    """
    Args:
      location: current directory or file

    Returns:
        parent directory absolute or relative path
    """
    _path = os.path.abspath if abspath else os.path.relpath
    return _path(f_join(location, os.pardir))


def md5_checksum(*fpath):
    """
    File md5 signature
    """
    hash_md5 = hashlib.md5()
    with open(f_join(*fpath), "rb") as f:
        for chunk in iter(lambda: f.read(65536), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()


def create_tar(fsrc, output_tarball, include=None, ignore=None, compress_mode="gz"):
    """
    Args:
        fsrc: source file or folder
        output_tarball: output tar file name
        compress_mode: "gz", "bz2", "xz" or "" (empty for uncompressed write)
        include: include pattern, will trigger copy to temp directory
        ignore: ignore pattern, will trigger copy to temp directory
    """
    fsrc, output_tarball = f_expand(fsrc), f_expand(output_tarball)
    assert compress_mode in ["gz", "bz2", "xz", ""]
    src_base = os.path.basename(fsrc)

    tempdir = None
    if include or ignore:
        tempdir = tempfile.mkdtemp()
        tempdest = f_join(tempdir, src_base)
        f_copy(fsrc, tempdest, include=include, ignore=ignore)
        fsrc = tempdest

    with tarfile.open(output_tarball, "w:" + compress_mode) as tar:
        tar.add(fsrc, arcname=src_base)

    if tempdir:
        f_remove(tempdir)


def extract_tar(source_tarball, output_dir=".", members=None):
    """
    Args:
        source_tarball: extract members from archive
        output_dir: default to current working dir
        members: must be a subset of the list returned by getmembers()
    """
    source_tarball, output_dir = f_expand(source_tarball), f_expand(output_dir)
    with tarfile.open(source_tarball, "r:*") as tar:
        tar.extractall(output_dir, members=members)


def move_with_backup(*fpath, suffix=".bak"):
    """
    Ensures that a path is not occupied. If there is a file, rename it by
    adding @suffix. Resursively backs up everything.

    Args:
        fpath: file path to clear
        suffix: Add to backed up files (default: {'.bak'})
    """
    fpath = str(f_join(*fpath))
    if os.path.exists(fpath):
        move_with_backup(fpath + suffix)
        shutil.move(fpath, fpath + suffix)


def insert_before_ext(name, insert):
    """
    log.txt -> log.ep50.txt
    """
    name, ext = os.path.splitext(name)
    return name + insert + ext


def timestamp_file_name(fname):
    timestr = datetime.now().strftime("_%H-%M-%S_%m-%d-%y")
    return insert_before_ext(fname, timestr)


def get_file_lock(*fpath, timeout: int = 15, logging_level="critical"):
    """
    NFS-safe filesystem-backed lock. `pip install flufl.lock`
    https://flufllock.readthedocs.io/en/stable/apiref.html

    Args:
        fpath: should be a path on NFS so that every process can see it
        timeout: seconds
    """
    from flufl.lock import Lock

    logging.getLogger("flufl.lock").setLevel(logging_level.upper())
    return Lock(f_join(*fpath), lifetime=timeout)


def load_pickle(*fpaths):
    with open(f_join(*fpaths), "rb") as fp:
        return pickle.load(fp)


def dump_pickle(data, *fpaths):
    with open(f_join(*fpaths), "wb") as fp:
        pickle.dump(data, fp)


def load_text(*fpaths, by_lines=False):
    with open(f_join(*fpaths), "r") as fp:
        if by_lines:
            return fp.readlines()
        else:
            return fp.read()


def load_text_lines(*fpaths):
    return load_text(*fpaths, by_lines=True)


def dump_text(s, *fpaths):
    with open(f_join(*fpaths), "w") as fp:
        fp.write(s)


def dump_text_lines(lines: list[str], *fpaths, add_newline=True):
    with open(f_join(*fpaths), "w") as fp:
        for line in lines:
            print(line, file=fp, end="\n" if add_newline else "")

class WithEmpty:
    def __enter__(self):
        pass
 
    def __exit__(self, *args):
        pass


# aliases to be consistent with other load_* and dump_*
pickle_load = load_pickle
pickle_dump = dump_pickle
text_load = load_text
read_text = load_text
read_text_lines = load_text_lines
write_text = dump_text
write_text_lines = dump_text_lines
text_dump = dump_text


In [6]:
## json utils
import json
import re
from typing import Any, Dict, Union
# from .file_utils import f_join


def json_load(*file_path, **kwargs):
    file_path = f_join(file_path)
    with open(file_path, "r") as fp:
        return json.load(fp, **kwargs)


def json_loads(string, **kwargs):
    return json.loads(string, **kwargs)


def json_dump(data, *file_path, **kwargs):
    file_path = f_join(file_path)
    with open(file_path, "w") as fp:
        json.dump(data, fp, **kwargs)


def json_dumps(data, **kwargs):
    """
    Returns: string
    """
    return json.dumps(data, **kwargs)


# ---------------- Aliases -----------------
# add aliases where verb goes first, json_load -> load_json
load_json = json_load
loads_json = json_loads
dump_json = json_dump
dumps_json = json_dumps


def extract_char_position(error_message: str) -> int:
    """Extract the character position from the JSONDecodeError message.
    Args:
        error_message (str): The error message from the JSONDecodeError
          exception.
    Returns:
        int: The character position.
    """
    import re

    char_pattern = re.compile(r"\(char (\d+)\)")
    if match := char_pattern.search(error_message):
        return int(match[1])
    else:
        raise ValueError("Character position not found in the error message.")


def add_quotes_to_property_names(json_string: str) -> str:
    """
    Add quotes to property names in a JSON string.
    Args:
        json_string (str): The JSON string.
    Returns:
        str: The JSON string with quotes added to property names.
    """

    def replace_func(match):
        return f'"{match.group(1)}":'

    property_name_pattern = re.compile(r"(\w+):")
    corrected_json_string = property_name_pattern.sub(replace_func, json_string)

    try:
        json.loads(corrected_json_string)
        return corrected_json_string
    except json.JSONDecodeError as e:
        raise e


def balance_braces(json_string: str) -> str:
    """
    Balance the braces in a JSON string.
    Args:
        json_string (str): The JSON string.
    Returns:
        str: The JSON string with braces balanced.
    """

    open_braces_count = json_string.count("{")
    close_braces_count = json_string.count("}")

    while open_braces_count > close_braces_count:
        json_string += "}"
        close_braces_count += 1

    while close_braces_count > open_braces_count:
        json_string = json_string.rstrip("}")
        close_braces_count -= 1

    try:
        json.loads(json_string)
        return json_string
    except json.JSONDecodeError as e:
        raise e


def fix_invalid_escape(json_str: str, error_message: str) -> str:
    while error_message.startswith("Invalid \\escape"):
        bad_escape_location = extract_char_position(error_message)
        json_str = json_str[:bad_escape_location] + json_str[bad_escape_location + 1 :]
        try:
            json.loads(json_str)
            return json_str
        except json.JSONDecodeError as e:
            error_message = str(e)
    return json_str


def correct_json(json_str: str) -> str:
    """
    Correct common JSON errors.
    Args:
        json_str (str): The JSON string.
    """

    try:
        json.loads(json_str)
        return json_str
    except json.JSONDecodeError as e:
        error_message = str(e)
        if error_message.startswith("Invalid \\escape"):
            json_str = fix_invalid_escape(json_str, error_message)
        if error_message.startswith(
            "Expecting property name enclosed in double quotes"
        ):
            json_str = add_quotes_to_property_names(json_str)
            try:
                json.loads(json_str)
                return json_str
            except json.JSONDecodeError as e:
                error_message = str(e)
        if balanced_str := balance_braces(json_str):
            return balanced_str
    return json_str


def fix_and_parse_json(
    json_str: str, try_to_fix_with_gpt: bool = True
) -> Union[str, Dict[Any, Any]]:
    """Fix and parse JSON string"""
    try:
        json_str = json_str.replace("\t", "")
        return json.loads(json_str)
    except json.JSONDecodeError as _:  # noqa: F841
        json_str = correct_json(json_str)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as _:  # noqa: F841
            pass
    # Let's do something manually:
    # sometimes GPT responds with something BEFORE the braces:
    # "I'm sorry, I don't understand. Please try again."
    # {"text": "I'm sorry, I don't understand. Please try again.",
    #  "confidence": 0.0}
    # So let's try to find the first brace and then parse the rest
    #  of the string
    try:
        brace_index = json_str.index("{")
        json_str = json_str[brace_index:]
        last_brace_index = json_str.rindex("}")
        json_str = json_str[: last_brace_index + 1]
        return json.loads(json_str)
    except json.JSONDecodeError as e:  # noqa: F841
        # if try_to_fix_with_gpt:
        #     print(
        #         "Warning: Failed to parse AI output, attempting to fix."
        #         "\n If you see this warning frequently, it's likely that"
        #         " your prompt is confusing the AI. Try changing it up"
        #         " slightly."
        #     )
        #     # Now try to fix this up using the ai_functions
        #     ai_fixed_json = fix_json(json_str, JSON_SCHEMA)
        #
        #     if ai_fixed_json != "failed":
        #         return json.loads(ai_fixed_json)
        #     else:
        #         # This allows the AI to react to the error message,
        #         #   which usually results in it correcting its ways.
        #         print("Failed to fix ai output, telling the AI.")
        #         return json_str
        # else:
        raise e


# def fix_json(json_str: str, schema: str) -> str:
#     """Fix the given JSON string to make it parseable and fully complient with the provided schema."""
#
#     # Try to fix the JSON using gpt:
#     function_string = "def fix_json(json_str: str, schema:str=None) -> str:"
#     args = [f"'''{json_str}'''", f"'''{schema}'''"]
#     description_string = (
#         "Fixes the provided JSON string to make it parseable"
#         " and fully complient with the provided schema.\n If an object or"
#         " field specified in the schema isn't contained within the correct"
#         " JSON, it is ommited.\n This function is brilliant at guessing"
#         " when the format is incorrect."
#     )
#
#     # If it doesn't already start with a "`", add one:
#     if not json_str.startswith("`"):
#         json_str = "```json\n" + json_str + "\n```"
#     result_string = call_ai_function(
#         function_string, args, description_string, model=cfg.fast_llm_model
#     )
#     if cfg.debug:
#         print("------------ JSON FIX ATTEMPT ---------------")
#         print(f"Original JSON: {json_str}")
#         print("-----------")
#         print(f"Fixed JSON: {result_string}")
#         print("----------- END OF FIX ATTEMPT ----------------")
#
#     try:
#         json.loads(result_string)  # just check the validity
#         return result_string
#     except:  # noqa: E722
#         # Get the call stack:
#         # import traceback
#         # call_stack = traceback.format_exc()
#         # print(f"Failed to fix JSON: '{json_str}' "+call_stack)
#         return "failed"


In [7]:
# from lego_prover.utils import load_json
# load miniF2F tasks and resume from the checkpoint
miniF2F_tasks = mp.Queue()
problem_names = []
if resume:
    if os.path.exists(f"{ckpt_dir}/curriculum/completed_tasks.json"):
        completed_tasks = load_json(
            f"{ckpt_dir}/curriculum/completed_tasks.json")
    if os.path.exists(f"{ckpt_dir}/curriculum/failed_tasks.json"):
        failed_tasks =load_json(f"{ckpt_dir}/curriculum/failed_tasks.json")
    print("Current progress: ", len(completed_tasks) + len(set(failed_tasks)))
else:
    completed_tasks = []
    failed_tasks = []
for name in os.listdir(f"data/full_data/{data_split}"):
    path = os.path.join(f"data/full_data/{data_split}", name)
    context = load_json(path)
    problem_names.append((path, len(context["informal_proof"])))
problem_names = sorted(problem_names, key=lambda x: x[1])
problem_names = [pn[0] for pn in problem_names]
problem_names = problem_names * number_of_prover_attempts     # 10 * 20 = 200 sketch
for pn in problem_names:
    if pn in completed_tasks:
        continue
    if pn in failed_tasks:
        failed_tasks.remove(pn)
        continue
    miniF2F_tasks.put(pn)
print(f"Sketch to finish: {miniF2F_tasks.qsize()}")



Sketch to finish: 24400


In [8]:
# setup multiprocessing logger
start_time = datetime.now(pytz.timezone(
    'Asia/Shanghai')).strftime("%Y%m%d_%H%M%S")

os.makedirs(f'logs/prover/{start_time}_logs', exist_ok=True)
for rank in range(number_of_prover_processes):
    logger = logging.getLogger(f'prover-{rank}')
    handler = logging.FileHandler(
        f"logs/prover/{start_time}_logs/rank_{rank}.log")
    formatter = logging.Formatter(
        "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)

os.makedirs(f'logs/evolver/{start_time}_logs', exist_ok=True)
for evolver_rank in range(number_of_evolver_processes):
    evolver_rank += number_of_prover_processes
    logger = logging.getLogger(f'evolver-{evolver_rank}')
    handler = logging.FileHandler(
        f"logs/evolver/{start_time}_logs/rank_{evolver_rank}.log")
    formatter = logging.Formatter(
        "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)


In [9]:
## Subprocessing

import json
import os
import time
import re
import warnings
from typing import List

import psutil
import subprocess
import logging
import threading

# import lego_prover.utils as U


class SubprocessMonitor:
    def __init__(
        self,
        commands: List[str],
        name: str,
        ready_match: str = r".*",
        log_path: str = "logs",
        callback_match: str = r"^(?!x)x$",  # regex that will never match
        callback: callable = None,
        finished_callback: callable = None,
        cwd: str = os.path.expanduser("~"),
        server_port: int = -1,
    ):
        self.commands = commands
        self.server_port = server_port
        start_time = time.strftime("%Y%m%d_%H%M%S")
        self.name = name
        if name == "isabelle_server":
            os.makedirs(f'logs/{name}/{start_time}_logs', exist_ok=True)
            self.logger = logging.getLogger(f'{name}-{server_port}')
            handler = logging.FileHandler(f"logs/{name}/{start_time}_logs/rank_{server_port}.log")
        else:
            self.logger = logging.getLogger(name)
            handler = logging.FileHandler(f_join(log_path, f"{start_time}.log"))
        formatter = logging.Formatter(
            "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
        )
        handler.setFormatter(formatter)
        self.logger.addHandler(handler)
        self.logger.setLevel(logging.INFO)
        self.process = None
        self.ready_match = ready_match
        self.ready_event = None
        self.ready_line = None
        self.callback_match = callback_match
        self.callback = callback
        self.finished_callback = finished_callback
        self.thread = None
        self.cwd = cwd

    def _start(self):
        self.logger.info(f"Starting subprocess with commands: {self.commands}")

        self.process = psutil.Popen(
            self.commands,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            stdin=subprocess.PIPE,
            universal_newlines=True,
            cwd=self.cwd
        )
        print(f"Subprocess {self.name} started with PID {self.process.pid}.")
        for line in iter(self.process.stdout.readline, ""):
            self.logger.info(line.strip())
            if re.search(self.ready_match, line):
                self.ready_line = line
                self.logger.info("Subprocess is ready.")
                self.ready_event.set()
                if "chroma" in self.name:
                    break
            if re.search(self.callback_match, line):
                self.callback()
        if not self.ready_event.is_set():
            self.ready_event.set()
            warnings.warn(f"Subprocess {self.name} failed to start.")
        if self.finished_callback:
            self.finished_callback()

    def run(self):
        self.ready_event = threading.Event()
        self.ready_line = None
        self.thread = threading.Thread(target=self._start)
        self.thread.start()
        self.ready_event.wait()

    def stop(self):
        self.logger.info("Stopping subprocess.")
        if self.process and self.process.is_running():
            self.process.terminate()
            self.process.wait()
    
    def terminate(self):
        parent = psutil.Process(self.process.pid)
        for child in parent.children(recursive=True):  # or parent.children() for recursive=False
            child.kill()
        parent.kill()

    def run_action(self, inputs):
        self.logger.info(f"Input: {inputs}")
        self.process.stdin.write(inputs + '\n')
        self.process.stdin.flush()

        for line in iter(self.process.stdout.readline, ""):
            self.logger.info(line)
            if line.startswith('{"error'):
                return json.loads(line)

    @property
    def is_running(self):
        if self.process is None:
            return False
        return self.process.is_running()


In [10]:
## Chroma
import json
import os
# import lego_prover.utils as U
# from .process_monitor import SubprocessMonitor
import time


class ChromaBridge:
    def __init__(
        self,
        ckpt_path="ckpt",
        resume=False,
        request_timeout=600,
        log_path="./logs",
    ):
        self.ckpt_path = ckpt_path
        self.resume = "True" if resume else "False"
        self.request_timeout = request_timeout
        self.log_path = log_path
        self.chroma_server = self.get_chroma_process()
        self.chroma_server.run()
        
        # wait for isabelle server to run
        time.sleep(3)

    def get_chroma_process(self):
        f_mkdir(self.log_path, "chromadb")
        return SubprocessMonitor(
            commands=[
                "python",
                "chroma_worker.py",
                "--ckpt_path",
                self.ckpt_path,
                "--resume",
                self.resume
            ],
            name="chroma_worker",
            ready_match=r"Chroma worker is ready.",
            log_path=f_join(self.log_path, "chromadb"),
            cwd=os.path.abspath("lego_prover/env/")
        )

    def run_cmd(self, cmd):
        cmd = json.dumps(cmd)
        return self.chroma_server.run_action(cmd)



In [11]:


from __future__ import annotations
import random
import time
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.openai import embed_with_retry
from langchain.chat_models.openai import _create_retry_decorator
from langchain.schema import LLMResult, AIMessage, HumanMessage, SystemMessage, ChatGeneration
# from openai_key import *




In [12]:
import os

from typing import List, Optional, Any

import numpy as np
import openai
import tiktoken

# from ananke.llm.thudm import ZhiPu
from ananke.llm.azure import Azure
from ananke.llm.ernie import Ernie

In [13]:


class LLMMixture:
    def __init__(self, model_name, temperature, request_timeout) -> None:
        self.encoder = tiktoken.encoding_for_model("gpt-4")
        self.model_name = model_name
        self.temperature = temperature
        self.request_timeout = request_timeout

        if 'gpt-3.5' in self.model_name:
            self.azure_openai_model = Azure(chat_model_name="Ananke3-1106-US-WEST")
        elif 'gpt-4' in self.model_name:
            self.azure_openai_model = Azure(chat_model_name="Ananke4-1106-US-WEST")
        else:
            self.azure_openai_model = Azure(chat_model_name="Ananke3-1106-US-WEST")
    
    def query(self, langchain_msgs, llm_type="short", n=1, temperature=None, max_tokens=None):
        success = False
        max_retry = 50
        messages = []
        for msg in langchain_msgs:
            if isinstance(msg, SystemMessage):
                messages.append({"role": "system", "content": msg.content})
            if isinstance(msg, HumanMessage):
                messages.append({"role": "user", "content": msg.content})
        while max_retry > 0:
            try:
                if temperature is None:
                    temperature = self.temperature
                response = self.azure_openai_model.client.chat.completions.create(
                            model=self.azure_openai_model.chat_model, messages=messages,
                            temperature=temperature
                            )
                # response = openai.ChatCompletion.create(
                #     model=llm_model,
                #     messages=messages,
                #     temperature=temperature,
                #     n=n,
                #     api_key=api_key[0],
                #     organization=api_key[1],
                #     max_tokens=max_tokens,
                # )
                # print("ckpt in 2")
            except openai.error.RateLimitError:
                print(".", end="", flush=True)
                time.sleep(0.1)
            except openai.error.APIConnectionError as e:
                time.sleep(random.randint(1,30))
                print(f"Openai Connection{e}")
                max_retry -= 1
            except openai.error.APIError as e:
                time.sleep(random.randint(1,30))
                if 'Bad gateway. {"error":{"code":502,"message":"Bad gateway."' in str(e):
                    print("-", end="", flush=True)
                else:
                    print(f"APIError了: {e}")
                max_retry -= 1
            except Exception as e:
                time.sleep(random.randint(1,30))
                print(f"Exception 了:{e}")
                max_retry -= 1
            else:
                success = True
                break
        if success:
            if n == 1:
                res = response.get("choices")[0]["message"]["content"]
                return res
            else:
                res = []
                for ix in range(n):
                    res.append(response.get("choices")[ix]["message"]["content"])
                return res
        else:
            return ""

    def __call__(self, messages, temperature=None, max_tokens=1024, n=1) -> Any:
        word_count = 0
        for msg in messages:
            word_count += len(self.encoder.encode(msg.content))
        if "gpt-4" in self.model_name:
            if word_count < 7000:
                results = self.query(messages, "short", temperature=temperature, n=n)
            else:
                assert False, f"query too long, with {word_count} token in total" 
        else:
            if word_count < 3500:
                results = self.query(messages, "short", temperature=temperature, n=n)
            elif word_count < (16385 - 2100):
                results = self.query(messages, "long",  temperature=temperature, max_tokens=max_tokens, n=n)
            else:
                assert False, f"query too long, with {word_count} token in total" 
        
        if n==1:
            return AIMessage(content=results)
        else:
            ret_messages = []
            for res in results:
                ret_messages.append(AIMessage(content=res))
            return ret_messages
            
    
    def generate(self, batch_message, slow_mode=False, temperature=None, max_tokens=1024):
        if slow_mode is False:
            # print("ckpt 1")
            n = len(batch_message)
            word_count = 0
            messages = batch_message[0]
            for msg in messages:
                word_count += len(self.encoder.encode(msg.content))
            # print(f"ckpt 2 {word_count}")
            if "gpt-4" in self.model_name:
                if word_count < 7000:
                    results = self.query(messages, "short", n=n, temperature=temperature, max_tokens=max_tokens)
                else:
                    assert False, f"query too long, with {word_count} token in total" 
            else:
                if word_count < 3500:
                    results = self.query(messages, "short", n=n, temperature=temperature, max_tokens=max_tokens)
                elif word_count < 15000:
                    results = self.query(messages, "long", n=n, temperature=temperature, max_tokens=max_tokens)
                else:
                    assert False, f"query too long, with {word_count} token in total" 
            generations = []
            for res in results:
                generations.append([ChatGeneration(message=AIMessage(content=res))])
            # print(f"Here successful with {len(results)}")
            return LLMResult(generations=generations)
        else:
            results = []
            for messages in batch_message:
                word_count = 0
                messages = batch_message[0]
                for msg in messages:
                    word_count += len(self.encoder.encode(msg.content))
                if word_count < 7000:
                    res = self.query(messages, "short")
                else:
                    res = self.query(messages, "long")
                results.append(res)
            generations = []
            for res in results:
                generations.append([ChatGeneration(text=res)])
            return LLMResult(generations=generations)




In [15]:
from copy import copy
import os
import random
import re
import time

# import lego_prover.utils as U
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# from lego_prover.prompts import load_prompt

# from lego_prover.utils.langchain_utils import LLMMixture

In [17]:
import pkg_resources
# import lego_prover.utils as U 


def load_prompt(prompt):
    package_path = pkg_resources.resource_filename("lego_prover", "")
    return load_text(f"{package_path}/prompts/{prompt}.txt")

def load_context(problem_path):
    return load_json(problem_path)

In [18]:


class ActionAgent:
    def __init__(
        self,
        logger=None,
        model_name="gpt-3.5-turbo",
        temperature=0,
        request_timeout=120,
        ckpt_dir="ckpt",
    ):
        self.logger = logger
        self.ckpt_dir = ckpt_dir
        f_mkdir(f"{ckpt_dir}/action")
        self.llm = LLMMixture(
            model_name=model_name,
            temperature=temperature,
            request_timeout=request_timeout,
        )

        # load decomposer examples:
        self.decomposer_examples = {}
        for file in os.listdir("data/decomposer_examples"):
            with open(os.path.join("data/decomposer_examples", file), "r") as f:
                text = f.read()
            self.decomposer_examples[file[:-4]] = text
        
        self.formalizer_examples = {}
        for file in os.listdir("data/formalizer_examples"):
            with open(os.path.join("data/formalizer_examples", file), "r") as f:
                text = f.read()
            self.formalizer_examples[file[:-4]] = text
    
    def retrieved_example_skills(self, retrieved_skills):
        random.shuffle(retrieved_skills)
        prompt_examples = []
        for ix, skills in enumerate(retrieved_skills):
            skill_code = skills["code"]
            prompt_example = f"""###### useful skill {ix+1}: ######
```isabelle
{skill_code}
```
"""
            prompt_examples.append(prompt_example)
        
        example_programmes = "\n\n".join(prompt_examples)
        return example_programmes
    
    def decomposer(self, context):
        system_prompt_template = load_prompt("decomposer")
        system_message = SystemMessage(content=system_prompt_template)

        human_prompt_template = load_prompt("decomposer_human")
        human_prompt_template = HumanMessagePromptTemplate.from_template(human_prompt_template)

        # post-process in-context-learning examples
        decomposer_examples = copy(self.decomposer_examples)
        if context["problem_name"] in decomposer_examples:
            decomposer_examples.pop(context["problem_name"])
        icl_examples = random.sample(list(decomposer_examples.values()), 3)
        icl_examples = "\n\n####################\n\n".join(icl_examples)

        context["informal_statement"] = context["informal_statement"].replace("\n", ' ').strip()
        context["informal_proof"] = context["informal_proof"].replace("\n", " ").strip()

        human_message = human_prompt_template.format(
            examples=icl_examples,
            informal_statement=context["informal_statement"],
            informal_proof=context["informal_proof"],
            formal_statement=context["formal_statement"],
        )

        conversation = {
            "sys0":  system_message.content,
            "human0": human_message.content,
        }

        self.logger.info(
            f"****decomposer system message****\n{system_message.content}"
        )

        self.logger.info(
            f"****decomposer human message****\n{human_message.content}"
        )

        n_retry = 3
        informal_proof = context["informal_proof"]
        skill_requests = []
        while n_retry > 0:
            try:
                ai_message = self.llm([system_message, human_message], temperature=0)
                self.logger.info(
                    f"****decomposer ai message****\n{ai_message.content}"
                )
                conversation[f"ai{3-n_retry}"] = ai_message.content
                message = ai_message.content
                if "####################" in message:
                    message = message[:message.index("####################")]
                # Extracting Error Analysis content
                informal_proof = re.search(r'## Structured informal proof\n(.*?)\n\n#', message, re.DOTALL).group(1).strip()

                # Extracting each skill request's name and its content
                skill_requests = re.findall(r"```isabelle\n(.*?)\n```", message, re.DOTALL)
                break
            except AssertionError as e:
                if "query too long" in str(e):
                    self.logger.warn(str(e))
                    break
            except Exception as e:
                self.logger.info(f"Error occur in decomposer: {str(e)}")
                n_retry -= 1
                examples = random.sample(list(decomposer_examples.values()), 3)
                examples = "\n\n####################\n\n".join(examples)
                human_message = human_prompt_template.format(
                    examples=examples,
                    informal_statement=context["informal_statement"],
                    informal_proof=context["informal_proof"],
                    formal_statement=context["formal_statement"],
                )
                time.sleep(5)
        ret_request = []
        for skill in skill_requests:
            if "N/A" in skill:
                continue
            ret_request.append(skill)

        if len(ret_request) > 5:
            self.logger.info(f"skill request more than 5, with len {len(ret_request)}")
            ret_request = random.sample(ret_request, 5)

        return informal_proof, ret_request, conversation

    def critic(self, context, code_last_round=None, error_last_round=None):
        system_prompt_template = load_prompt("critic_request")
        system_prompt_template = SystemMessagePromptTemplate.from_template(system_prompt_template)
        system_message = system_prompt_template.format(examples="")

        human_prompt_template = load_prompt("critic_request_human")
        human_prompt_template = HumanMessagePromptTemplate.from_template(human_prompt_template)

        if code_last_round is None:
            code_last_round = "No code from last round..."
        else:
            code_last_round = code_last_round.split('\n')
            new_code = []
            for ix, line in enumerate(code_last_round):
                line = f"#{ix+1} " + line
                new_code.append(line)
            code_last_round = "\n".join(new_code)
        
        if error_last_round is None:
            error_last_round = "No error from last round..."

        human_message = human_prompt_template.format(
            code=code_last_round,
            error=error_last_round,
        )

        # self.logger.info(
        #     f"****critic agent system message****\n{system_message.content}"
        # )

        self.logger.info(
            f"****critic agent human message****\n{human_message.content}"
        )

        n_retry = 3
        error_analysis = "No error analysis..."
        skill_requests = []
        while n_retry > 0:
            try:
                ai_message = self.llm([system_message, human_message])
                self.logger.info(
                    f"****critic agent ai message****\n{ai_message.content}"
                )
                message = ai_message.content
                # Extracting Error Analysis content
                error_analysis = re.search(r'# Error analysis:\n(.*?)\n\n#', message, re.DOTALL).group(1).strip()

                # Extracting each skill request's name and its content
                skill_requests = re.findall(r'## Skill \d+: ([\w_]+)\n```isabelle\n(.*?)\n```', message, re.DOTALL)
                break
            except AssertionError as e:
                if "query too long" in str(e):
                    self.logger.warn(str(e))
                    break
            except Exception as e:
                self.logger.info(f"Error occur in auto_formal_pre: {str(e)}")
                n_retry -= 1
                time.sleep(5)

        return error_analysis, skill_requests
    
    def render_formalizer_system_message(self):
        system_template = load_prompt("formalizer")
        return SystemMessage(content=system_template)
    
    def render_formalizer_human_message(
        self,
        skills,
        context,
        informal_proof=None,
        n_example=3,
    ) -> HumanMessage:
        human_prompt_template = load_prompt("formalizer_human")
        human_prompt_template = HumanMessagePromptTemplate.from_template(human_prompt_template)

        formalizer_examples = copy(self.formalizer_examples)
        if context["problem_name"] in formalizer_examples:
            formalizer_examples.pop(context["problem_name"])

        examples = random.sample(list(formalizer_examples.values()), n_example)
        examples = "\n\n####################\n\n".join(examples)
        context["informal_statement"] = context["informal_statement"].replace("\n", ' ').strip()
        context["informal_proof"] = context["informal_proof"].replace("\n", " ").strip()

        skills = self.retrieved_example_skills(skills)
        
        human_message = human_prompt_template.format(
            skill_examples = skills,
            examples=examples,
            informal_statement=context["informal_statement"],
            informal_proof=context["informal_proof"] if informal_proof is None else informal_proof,
            formal_statement=context["formal_statement"],
        )

        return human_message


    def render_human_message(
        self, 
        context, 
        code=None,
        error=None,
        error_analysis=None,
        informal_proof=None,
    ) -> HumanMessage:
        human_prompt_template = load_prompt("auto_formal2_human")
        human_prompt_template = HumanMessagePromptTemplate.from_template(human_prompt_template)

        if code is None:
            code = "No code from last round..."
        else:
            code = code.split('\n')
            new_code = []
            for ix, line in enumerate(code):
                line = f"#{ix+1} " + line
                new_code.append(line)
            code = "\n".join(new_code)
        
        if error is None:
            error = "No error from last round..."
        if error_analysis is None:
            error_analysis = "No analysis..."

        human_message = human_prompt_template.format(
            informal_statement=context["informal_statement"],
            informal_proof=context["informal_proof"] if informal_proof is None else informal_proof,
            formal_statement=context["formal_statement"],
            code_last_round=code,
            error_last_round=error,
            error_analysis=error_analysis,
        )

        return human_message

    def process_ai_message(self, message, context):
        assert isinstance(message, AIMessage)

        retry = 3
        error = None
        while retry > 0:
            try:
                code_pattern = re.compile(r"```(?:[i|I]sabelle)(.*?)```", re.DOTALL)
                text = message.content[message.content.index("# Formalized Code"):]
                code = "\n".join(code_pattern.findall(text)).strip()
                return code
            except Exception as e:
                retry -= 1
                error = e
                time.sleep(1)
        self.logger.info(f"Error parsing action response (before program execution): {error}")
        return False



In [27]:
import os
import re

import tiktoken

# import lego_prover.utils as U
# from lego_prover.env.chromas import ChromaBridge

# from lego_prover.utils.langchain_utils import LLMMixture

from difflib import SequenceMatcher, get_close_matches

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


class SkillManager:
    def __init__(
        self,
        rank = None,
        logger = None,
        ckpt_dir="ckpt",
        skill_manager_lock=WithEmpty(),
        chroma_bridge: ChromaBridge  = None
    ):
        self.rank = rank
        self.logger = logger
        self.skill_manager_lock = skill_manager_lock
        self.chroma_bridge = chroma_bridge
        f_mkdir(f"{ckpt_dir}/skill/code")
        f_mkdir(f"{ckpt_dir}/skill/history_problem")
        f_mkdir(f"{ckpt_dir}/skill/requests")
        f_mkdir(f"{ckpt_dir}/skill/description")
        f_mkdir(f"{ckpt_dir}/skill/vectordb")
        self.ckpt_dir = ckpt_dir
        self.encoder = tiktoken.encoding_for_model("gpt-4")
        with self.skill_manager_lock:
            self.sync_checkpoint()
    
    def sync_checkpoint(self):
        if os.path.exists(f"{self.ckpt_dir}/skill/skills.json"):
            self.skills = load_json(f"{self.ckpt_dir}/skill/skills.json")
        else:
            self.skills = {}
        if os.path.exists(f"{self.ckpt_dir}/skill/codes.json"):
            self.codes = load_json(f"{self.ckpt_dir}/skill/codes.json")
        else:
            self.codes = {}
        if os.path.exists(f"{self.ckpt_dir}/skill/skill_request.json"):
            self.skill_requests = load_json(f"{self.ckpt_dir}/skill/skill_request.json")
        else:
            self.skill_requests = {}
    
    def add_new_problem(self, problem_name, formal_statement):
        data = ("problem_add_text", {
                "add_text": formal_statement,
                "problem_name": problem_name,
        })
        output = self.chroma_bridge.run_cmd(data)
        assert output["error"] is None, "error is not None"
        print(output["output"])

    def add_new_request(self, problem_name, formal_statement, init_update_count=0):
        with self.skill_manager_lock:
            self.sync_checkpoint()

        exists_formal_statements = [value['formal_statement'] for value in self.skill_requests.values()]
        if len(get_close_matches(formal_statement, exists_formal_statements, n=1, cutoff=0.85)) != 0:
            return

        with self.skill_manager_lock:
            self.sync_checkpoint()
            request_name = f"request_{len(self.skill_requests)}"
            self.skill_requests[request_name] = {
                "request_name": request_name,
                "problem_name": problem_name,
                "formal_statement": formal_statement,
                "update_count": init_update_count,
            }
            

            data = ("request_add_text", {
                "add_text": formal_statement,
                "request_name": request_name,
            })
            
            assert self.chroma_bridge is not None
            output = self.chroma_bridge.run_cmd(data)
            if output["error"] is None:
                # print("There are",  output["output"], "code")
                assert output["output"] == len(
                    self.skill_requests
                ), ("requestdb is not synced with skill_request.json, "
                    f"there are {output['output']} in requestdb but {len(self.skill_requests)} in skill_request.json")
            
            dump_text(
                formal_statement, f"{self.ckpt_dir}/skill/requests/{request_name}.thy"
            )
            dump_json(self.skill_requests, f"{self.ckpt_dir}/skill/skill_request.json")
            self.logger.info(f"Added skill, marker:\n ```isabelle\n{formal_statement}```\n")      

    def add_new_skill(self, skill_name, description, marker, full_code, origin="", init_update_count=0):
        with self.skill_manager_lock:
            self.sync_checkpoint()

        exists_markers = [value['marker'] for value in self.skills.values()]
        if len(self.encoder.encode(marker)) > 650:
            return
        if len(get_close_matches(marker, exists_markers, n=1, cutoff=0.85)) != 0:
            return

        if not bool(re.match("^[a-zA-Z0-9_']+$", skill_name)):
            skill_name = f"skill_{len(self.skills)}"

        skill_name = skill_name.lower().strip().replace(" ", "_")
        if skill_name in self.skills:
            i = 2
            while f"{skill_name}V{i}" in self.skills:
                i += 1
            skill_name = f"{skill_name}V{i}"

        with self.skill_manager_lock:
            self.sync_checkpoint()

            self.skills[skill_name] = {
                "skill_name": skill_name,
                "marker": marker,
                "description": description,
                "full_code": full_code,
                "origin": origin,
                "update_count": init_update_count,
            }

            # add_text = f"code: {marker}, skill: {skill_name}, description: {description},"
            add_text = marker
            
            # use chroma bridge to add skill to the chromadb
            assert self.chroma_bridge is not None
            data = ("skill_add_text",{
                "skill_name": skill_name,
                "add_text": add_text,
            })
            output = self.chroma_bridge.run_cmd(data)
            if output["error"] is None:
                assert output["output"] == len(
                    self.skills
                ), ("vectordb is not synced with skill.json"
                    f"there are {output['output']} in skilldb but {len(self.skills)} in skills.json")
            
            dump_text(
                marker, f"{self.ckpt_dir}/skill/code/{skill_name}.thy"
            )
            dump_text(
                description,
                f"{self.ckpt_dir}/skill/description/{skill_name}.txt",
            )
            dump_json(self.skills, f"{self.ckpt_dir}/skill/skills.json")
            self.logger.info(f"Added skill, marker:\n ```isabelle\n{marker}```\nfull_code:\nisabelle\n{full_code}\n")

    def update_count(self, skill_name):
        with self.skill_manager_lock:
            self.sync_checkpoint()
            self.skills[skill_name]["update_count"] += 1
            dump_json(self.skills, f"{self.ckpt_dir}/skill/skills.json")
    
    def update_count_request(self, request_name):
        with self.skill_manager_lock:
            self.sync_checkpoint()
            self.skill_requests[request_name]["update_count"] += 1
            dump_json(self.skill_requests, f"{self.ckpt_dir}/skill/skill_request.json")

    def retrieve_skills(self, query, k):
        ret_skill = []
        k = min(len(self.skills), k)
        if k != 0:
            self.logger.info(f"Skill Manager retrieving for {k} skills")
            with self.skill_manager_lock:
                # query = f"informal statement: {context['informal_statement']}, informal proof: {context['informal_proof']}, formal_statement: {context['formal_statement']}"
                data = ("skill_query", {"query": query, "k": k})
                outputs = self.chroma_bridge.run_cmd(data)
                ret_skill_name = []
                if outputs["error"] is None:
                    ret_skill_name = outputs["output"]
                self.sync_checkpoint()
            self.logger.info(
                f"Skill Manager retrieved skills for query:\n ```\n"
                f"{query}\n```\n"
                f"{', '.join(ret_skill_name)}"
            )

            for skill_name in ret_skill_name:
                retrieved_skill = {
                    "skill": skill_name,
                    "description": self.skills[skill_name]["description"],
                    "code": self.skills[skill_name]["full_code"],
                    "marker": self.skills[skill_name]["marker"],
                }
                ret_skill.append(retrieved_skill)
        return ret_skill

    def retrieve_skills_with_context(self, context):
        ret_skill = []

        k = min(len(self.skills), 6)
        if k != 0:
            self.logger.info(f"Skill Manager retrieving for {k} skills")
            with self.skill_manager_lock:
                query = context['formal_statement']
                data = ("skill_query", {"query": query, "k": k})
                outputs = self.chroma_bridge.run_cmd(data)
                ret_skill_name = []
                if outputs["error"] is None:
                    ret_skill_name = outputs["output"]
                self.sync_checkpoint()
            self.logger.info(
                f"Skill Manager retrieved skills for query:\n ```\n"
                f"{query}\n```\n"
                f"{', '.join(ret_skill_name)}"
            )
        
            for skill_name in ret_skill_name:
                retrieved_skill = {
                    "skill": skill_name,
                    "description": self.skills[skill_name]["description"],
                    "code": self.skills[skill_name]["full_code"],
                    "marker": self.skills[skill_name]["marker"],
                }
                ret_skill.append(retrieved_skill)

        return ret_skill


In [24]:
from __future__ import annotations
import os

# import lego_prover.utils as U 
# from lego_prover.prompts import load_context
import multiprocessing as mp

import logging

class CurriculumAgent:
    def __init__(
        self,
        logger=None,
        ckpt_dir="ckpt",
        resume=False,
        miniF2F_tasks : mp.Queue = None,
        curriculum_task_type : str = "simple_curriculum",
        curriculum_agent_lock = WithEmpty()
    ):
        self.logger=logger
        self.miniF2F_tasks = miniF2F_tasks
        self.curriculum_task_type = curriculum_task_type
        self.curriculum_agent_lock = curriculum_agent_lock
        self.ckpt_dir = ckpt_dir
        f_mkdir(f"{ckpt_dir}/curriculum/vectordb")
        if resume:
            self.logger.info(f"Loading Curriculum Agent from {ckpt_dir}/curriculum")
            self.sync_checkpoint()
        else:
            self.completed_tasks = []
            self.failed_tasks = []
    
    def sync_checkpoint(self,):
        if os.path.exists(f"{self.ckpt_dir}/curriculum/completed_tasks.json"):
            self.completed_tasks = load_json(f"{self.ckpt_dir}/curriculum/completed_tasks.json")
        if os.path.exists(f"{self.ckpt_dir}/curriculum/failed_tasks.json"):
            self.failed_tasks = load_json(f"{self.ckpt_dir}/curriculum/failed_tasks.json")

    @property
    def easy_to_hard_curriculum(self):
        result = []
        for name in os.listdir("data/full_data/valid"):
            path = os.path.join("data/full_data/valid", name)
            context = load_json(path)
            result.append((path, len(context["informal_proof"])))
        result = sorted(result, key=lambda x: x[1])
        result = [x[0] for x in result]
        return result

    @property
    def progress(self):
        return len(self.completed_tasks)

    def propose_next_task(self, max_retries=5, idx=None):
        if self.curriculum_task_type == "example":
            filename = os.listdir("data/examples")[self.progress]
            task = filename[:-5]
            context = load_context(problem_name=os.path.join("data/examples", filename))
            return task, context
        elif self.curriculum_task_type == "simple_curriculum":
            assert idx is not None
            file_path = self.easy_to_hard_curriculum[idx]
            task = file_path
            context = load_context(file_path)
            return task, context
        elif self.curriculum_task_type == "queue_curriculum":
            while True:
                if self.miniF2F_tasks.qsize() == 0:
                    return "", None
                file_path = self.miniF2F_tasks.get()
                context = load_context(file_path)
                if file_path not in self.completed_tasks:
                    break
            return file_path, context
        else:
            raise NotImplementedError

    def get_task_retry_count(self, task):
        cnt = 0
        for t in self.failed_tasks:
            if t == task:
                cnt += 1
        return cnt

    def propose_next_manual_task(self):
        confirmed = False
        task = ""
        while not confirmed:
            task = input("Enter task: ")
            print(f"Task: {task}")
            confirmed = input("Confirm? (y/n)").lower() in ["y", ""]
        context = load_context(task)
        return task, context

    def update_exploration_progress(self, info):
        with self.curriculum_agent_lock:
            self.sync_checkpoint()

            task = info["task"]
            if info["success"]:
                self.logger.info(f"Completed task {task}.")
                self.completed_tasks.append(task)
            else:
                self.logger.info(
                    f"Failed to complete task {task}. Skipping to next task."
                )
                self.failed_tasks.append(task)

            # clean up tasks and dump to disk
            self.clean_up_tasks()

    def clean_up_tasks(self):
        updated_completed_tasks = []
        # record repeated failed tasks
        updated_failed_tasks = self.failed_tasks
        # dedup but keep order
        for task in self.completed_tasks:
            if task not in updated_completed_tasks:
                updated_completed_tasks.append(task)

        # remove completed tasks from failed tasks
        for task in updated_completed_tasks:
            while task in updated_failed_tasks:
                updated_failed_tasks.remove(task)

        self.completed_tasks = updated_completed_tasks
        self.failed_tasks = updated_failed_tasks

        # dump to json
        dump_json(
            self.completed_tasks, f"{self.ckpt_dir}/curriculum/completed_tasks.json"
        )
        dump_json(self.failed_tasks, f"{self.ckpt_dir}/curriculum/failed_tasks.json")


In [25]:
import os
import random
import re
import time
import multiprocessing as mp
import tiktoken
from lego_prover.env.isa_bridge import IsabelleEnv

# import lego_prover.utils as U

# from .agents import ActionAgent
# from .agents import CurriculumAgent
# from .agents import SkillManager
from langchain.schema import HumanMessage

import logging


class Prover:
    def __init__(
        self,
        rank: int = None,
        isabelle_path: str = None,
        server_port: int = 8000,
        model_name: str = "gpt-4",
        temperature: int = 0,
        action_agent_task_max_retries: int = 4,
        curriculum_task_type: str = "simple_curriculum",
        curriculum_agent_lock = WithEmpty(),
        skill_manager_lock = WithEmpty(),
        chroma_bridge = None,
        openai_api_request_timeout: int = 6000,
        ckpt_dir: str = "ckpt",
        resume: bool = False,
        miniF2F_tasks: mp.Queue = None,
    ):
        """
        Initializes a new instance of the Prover class.

        Args:
            rank (int): The rank of the prover process.
            isabelle_path (str): The path to the Isabelle directory.
            server_port (int): The port number for the server.
            model_name (str): The name of the OpenAI model to use.
            temperature (int): The temperature for sampling the LLM.
            action_agent_task_max_retries (int): The maximum number of retries for an action agent task.
            curriculum_task_type (str): The type of curriculum task to use.
            curriculum_agent_lock: The lock for the curriculum agent.
            skill_manager_lock: The lock for the skill manager.
            chroma_bridge: The ChromaBridge object for controlling the keyboard and mouse.
            openai_api_request_timeout (int): The timeout for OpenAI API requests.
            ckpt_dir (str): The directory for saving checkpoints.
            resume (bool): Whether to resume from the checkpoint.
            miniF2F_tasks (mp.Queue): The queue for miniF2F tasks.
        """

        # init env
        self.rank = rank
        self.logger = logging.getLogger(f'prover-{rank}')
        self.logger.info(f"lego_prover running in rank {rank}")
        self.model_name = model_name

        self.env = IsabelleEnv(
            logger=self.logger,
            isabelle_path=isabelle_path,
            server_port=server_port
        )
        self.action_agent_model_name = model_name
        self.tokenizer_encoder = tiktoken.encoding_for_model(
            self.action_agent_model_name)

        self.ckpt_dir = ckpt_dir
        self.temperature = temperature

        # init agents
        self.action_agent = ActionAgent(
            logger=self.logger,
            model_name=model_name,
            temperature=temperature,
            request_timeout=openai_api_request_timeout,
            ckpt_dir=ckpt_dir,
        )
        self.action_agent_task_max_retries = action_agent_task_max_retries
        self.curriculum_agent = CurriculumAgent(
            logger=self.logger,
            ckpt_dir=ckpt_dir,
            resume=resume,
            miniF2F_tasks=miniF2F_tasks,
            curriculum_task_type=curriculum_task_type,
            curriculum_agent_lock=curriculum_agent_lock,
        )
        self.skill_manager = SkillManager(
            rank=rank,
            logger=self.logger,
            ckpt_dir=ckpt_dir,
            skill_manager_lock=skill_manager_lock,
            chroma_bridge=chroma_bridge,
        )
        self.resume = resume

        # init variables for rollout
        self.action_agent_rollout_num_iter = -1
        self.task = None
        self.context = ""
        self.messages = None
        self.conversations = []
        self.last_events = None

    def _fill_skills(self, retrieved_skills, requested_skills, n_retrieved, n_requested, model_name):
        """
        Given the retrieved skills query by problem statement and requests, output `n_retrieved + n_requested`
        skill examples.
        """
        if model_name == "gpt-4":
            raise NotImplementedError

        skills = random.sample(retrieved_skills, min(n_retrieved, len(self.retrieved_skills))) + \
            random.sample(requested_skills, min(
                n_requested, len(self.requested_skills)))

        skill_names = [skill["skill"] for skill in skills]
        n_skill = n_retrieved + n_requested
        if len(skills) < n_skill:
            for s in requested_skills:
                if len(skills) == n_skill:
                    break
                if s["skill"] not in skill_names:
                    skills.append(s)
            for s in retrieved_skills:
                if len(skills) == n_skill:
                    break
                if s["skill"] not in skill_names:
                    skills.append(s)
        self.logger.info(f"There are {len(skills)} in total")
        return skills

    def reset(self, task, context, reset_env=True):
        self.context = context
        self.action_agent_rollout_num_iter = 0
        self.task = task
        if reset_env:
            self.env.reset()

        self.retrieved_skills = self.skill_manager.retrieve_skills_with_context(
            context=context)
        self.informal_proof, skill_requests, conversation = self.action_agent.decomposer(
            context=context,
        )
        self.info = {"decomposer_conversation": conversation}
        for request in skill_requests:
            request_name = self.env.get_lemma_name(request)
            self.logger.info(
                f"adding request: name: {request_name}, code: {request}")
            self.skill_manager.add_new_request(
                problem_name=task, formal_statement=request, init_update_count=0)

        # request skill
        self.requested_skills = []
        requested_skills_name = [s["skill"] for s in self.retrieved_skills]
        for skill_context in skill_requests:
            name = self.env.get_lemma_name(skill_context)
            query = f"code: {skill_context}, skill: {name}"
            requested_skill = self.skill_manager.retrieve_skills(query, 2)
            self.logger.info(
                f"Skill request query: {query} with result {requested_skill}")
            for rskill in requested_skill:
                if rskill["skill"] not in requested_skills_name:
                    requested_skills_name.append(rskill["skill"])
                    self.requested_skills.append(rskill)
        self.logger.info(
            f"There are {len(skill_requests)} with result of {len(self.requested_skills)} requested skill retrieved")

        system_message = self.action_agent.render_formalizer_system_message()
        skills = self._fill_skills(
            self.retrieved_skills, self.requested_skills, 0, 4, self.model_name)

        human_message = self.action_agent.render_formalizer_human_message(
            skills=skills, context=context, informal_proof=self.informal_proof, n_example=2
        )
        self.messages = [system_message, human_message]
        self.logger.info(
            f"****formalizer system message****\n{system_message.content}"
        )
        assert len(self.messages) == 2
        self.conversations = []
        self.history_messages = []
        return self.messages

    def close(self):
        self.env.close()

    def step(self):
        if self.action_agent_rollout_num_iter < 0:
            raise ValueError("Agent must be reset before stepping")
        skill_codes = []

        conversation = {"action_agent_sys0": self.messages[0].content,
                        "action_agent_human0": self.messages[1].content}
        # query model
        n_retry = 3
        while n_retry > 0:
            try:
                self.logger.info(
                    f"****formalizer human message****\n{self.messages[-1].content}"
                )
                ai_message = self.action_agent.llm(
                    self.messages, temperature=self.temperature, max_tokens=2000)
                conversation[f"action_agent_ai{3 - n_retry}"] = ai_message.content
                self.logger.info(
                    f"****formalizer ai message****\n{ai_message.content}")
                # text = ai_message.content[ai_message.content.index("# Formalized code"):]
                text = ai_message.content
                if "####################" in text:
                    text = text[:text.index("####################")]
                code_pattern = re.compile(
                    r"```(?:[i|I]sabelle)(.*?)```", re.DOTALL)
                parsed_result = "\n".join(code_pattern.findall(text)).strip()
                assert self.context["formal_statement"] in parsed_result, \
                    "Formal statement is not in the formal code generated"
                break
            except AssertionError as e:
                if "query too long" in str(e):
                    self.logger.info(str(e))
                    parsed_result = False
                    context_length = len(self.messages[1].content)
                    self.messages[1] = HumanMessage(
                        content=self.messages[1].content[int(context_length * 0.3):])
                    n_retry -= 1
                    conversation[f"formalizer{3 - n_retry}"] = self.messages[1].content
                else:
                    self.logger.info(f"parse failed with error: {str(e)}")
                    parsed_result = False
                    skills = self._fill_skills(
                        self.retrieved_skills, self.requested_skills, 0, 4, self.model_name)
                    human_message = self.action_agent.render_formalizer_human_message(
                        skills=skills, context=self.context, informal_proof=self.informal_proof, n_example=2
                    )
                    n_retry -= 1
                    conversation[f"formalizer{3 - n_retry}"] = self.messages[1].content
                    self.messages[1] = human_message
            except Exception as e:
                self.logger.info(f"parse failed with error: {str(e)}")
                parsed_result = False
                skills = self._fill_skills(
                    self.retrieved_skills, self.requested_skills, 0, 4, self.model_name)
                human_message = self.action_agent.render_formalizer_human_message(
                    skills=skills, context=self.context, informal_proof=self.informal_proof, n_example=2
                )
                n_retry -= 1
                conversation[f"formalizer{3 - n_retry}"] = self.messages[1].content
                self.messages[1] = human_message
                time.sleep(5)

        self.history_messages += [self.messages[1], ai_message]
        self.conversations.append(
            (self.messages[0].content,
             self.messages[1].content, ai_message.content)
        )
        if isinstance(parsed_result, str):
            self.logger.info("*******Parse success, verifying result*******")
            verified_result, parsed_result, skill_codes, requests = self.env.step(
                parsed_result, formal_statement=self.context["formal_statement"])
            self.logger.info(f'Success: {verified_result["success"]}')
            self.logger.info(f'Error: {verified_result["reason"]}')
            if len(requests) > 10:
                self.logger.warn(
                    f"Too many request! with {len(requests)} in total")
                requests = random.sample(requests, 10)
            for req in requests:
                self.skill_manager.add_new_request(
                    problem_name=self.task,
                    formal_statement=req,
                    init_update_count=-3,
                )
        else:
            assert isinstance(parsed_result, bool)
            self.logger.warn(f"{parsed_result} Trying again!")
            verified_result = {"success": False}
        assert len(self.messages) == 2
        self.action_agent_rollout_num_iter += 1
        done = True
        info = {
            "task": self.task,
            "success": verified_result["success"],
            "conversations": self.conversations,
            "code": parsed_result,
            "context": self.context,
            "verified_result": verified_result,
            "action_agent_conversation": conversation
        }
        self.info.update(info)
        return self.messages, 0, done, self.info, skill_codes

    def rollout(self, *, task, context, reset_env=True, gt_formal_code=None):
        all_skill_codes = []
        self.reset(task=task, context=context, reset_env=reset_env)
        while True:
            messages, reward, done, info, skill_codes = self.step()
            all_skill_codes.extend(skill_codes)
            if done or gt_formal_code is not None:
                break

        if info["success"] is True:
            self.logger.info("########## Final result ##########")
            self.logger.info(f'{info["code"]}')
            self.logger.info("##################################")
        else:
            self.logger.info("########## Final result ##########")
            self.logger.info('sad!!!!!')
            self.logger.info("##################################")

        # -- deduplicate according to the marker
        if len(all_skill_codes) > 0:
            markers, full_codes = list(zip(*all_skill_codes))
            dedup_set = set()
            all_skill_codes = []
            for ix, marker in enumerate(markers):
                if marker in dedup_set:
                    continue
                dedup_set.add(marker)
                all_skill_codes.append([marker, full_codes[ix]])

        if len(all_skill_codes) > 10:
            self.logger.warn(
                f"There are {len(all_skill_codes)} to be added, it's problematic!! skipping...")
            all_skill_codes = random.sample(all_skill_codes, 10)

        # self.skill_manager.conclude_skills(context, info, all_skill_codes)
        for marker, full_code in all_skill_codes:
            code = f'''theory Scratch\n  imports Complex_Main\nbegin\n\n{full_code}\nend'''
            result, *_ = self.env.step(code, quick_check=True)
            if result is True:
                self.skill_manager.add_new_skill(
                    skill_name=self.env.get_lemma_name(marker),
                    description="",
                    marker=marker,
                    full_code=full_code,
                    origin=f"{task}_v{self.curriculum_agent.get_task_retry_count(task)}",
                    init_update_count=-1,
                )

        return messages, reward, done, info

    def learn(self, reset_env=True):
        while True:
            task, context = self.curriculum_agent.propose_next_task(
                max_retries=5)
            if task is None:
                break

            task_retried_cnt = self.curriculum_agent.get_task_retry_count(task)
            self.logger.info(
                f"Starting task {task} at {task_retried_cnt + 1} try and for at most {self.action_agent_task_max_retries} error correction"
            )
            try:
                messages, reward, done, info = self.rollout(
                    task=task,
                    context=context,
                    reset_env=reset_env,
                )
            except TabError as e:
                info = {
                    "task": task,
                    "success": False,
                }
                # reset isabelle
                self.env.reset(hard_reset=True)
                time.sleep(3)  # wait for prior programme to init
                # use red color background to print the error
                self.logger.info(
                    "Your last round rollout terminated due to error:")
                self.logger.info(f"\033[41m{e}")

            self.curriculum_agent.update_exploration_progress(info)
            self.logger.info(
                f"Completed tasks: {', '.join(self.curriculum_agent.completed_tasks)}"
            )
            self.logger.info(
                f"Failed tasks: {', '.join(self.curriculum_agent.failed_tasks)}"
            )

            n_complete_task = len(set(list(filter(
                lambda x: "data/full_data" in x, self.curriculum_agent.completed_tasks))))
            n_failed_task = len(set(list(filter(
                lambda x: "data/full_data" in x, self.curriculum_agent.failed_tasks))))

            self.logger.info(f"Number of completed tasks: {n_complete_task}")
            self.logger.info(f"Number of failed tasks: {n_failed_task}")
            self.logger.info(
                f"pass rate: {n_complete_task / (n_complete_task + n_failed_task)}")

            print(f"Success: {info['success']} - [{task}], try: {task_retried_cnt + 1},",
                  f"Progress: {n_complete_task + n_failed_task}/244,",
                  f"Sketch remaining: {self.curriculum_agent.miniF2F_tasks.qsize()}"
                  f"Pass rate: {n_complete_task / (n_complete_task + n_failed_task)}", flush=True)
            os.makedirs(f"{self.ckpt_dir}/json_logs", exist_ok=True)
            dump_json(
                info, f"{self.ckpt_dir}/json_logs/task_{context['problem_name']}_try{task_retried_cnt + 1}.json")

        return {
            "completed_tasks": self.curriculum_agent.completed_tasks,
            "failed_tasks": self.curriculum_agent.failed_tasks,
            "skills": self.skill_manager.skills,
        }


In [20]:
def run_prover(rank, tasks, skill_manager_lock, curriculum_agent_lock, chroma_bridge):
    server_port = 8051 + rank

    prover = Prover(
        rank=rank,
        isabelle_path=isabelle_path,
        server_port=server_port,
        model_name=model_name,
        skill_manager_lock=skill_manager_lock,
        action_agent_task_max_retries=1,
        curriculum_task_type="queue_curriculum",
        curriculum_agent_lock=curriculum_agent_lock,
        resume=resume,
        temperature=temperature,
        miniF2F_tasks=tasks,
        ckpt_dir=ckpt_dir,
        chroma_bridge=chroma_bridge,
    )
    prover.learn()

def run_evolver(rank, skill_manager_lock, chroma_bridge):
    server_port = 8011 + rank
    evolver = Evolver(
        rank=rank,
        isabelle_path=isabelle_path,
        ckpt_dir=ckpt_dir,
        server_port=server_port,
        data_split=data_split,
        skill_manager_lock=skill_manager_lock,
        model_name=model_name,
        temperature=temperature,
        chroma_bridge=chroma_bridge
    )
    evolver.evolve()



In [21]:
# processes = []
# skill_manager_lock = mp.Lock()
# curriculum_agent_lock = mp.Lock()
# chroma_bridge = ChromaBridge(ckpt_path=ckpt_dir, resume=resume)

Subprocess chroma_worker started with PID 2433467.


/tmp/ipykernel_2432657/2043160899.py:82: UserWarning: Subprocess chroma_worker failed to start.
  warnings.warn(f"Subprocess {self.name} failed to start.")


In [ ]:
rank = 0
run_prover(rank, miniF2F_tasks, skill_manager_lock, curriculum_agent_lock, chroma_bridge)

Subprocess isabelle_server started with PID 2435733.


/workspace/huangyongfeng/ananke/example/pipeline/math_experiment/LEGO-Prover/lego_prover/env/process_monitor.py:80: UserWarning: Subprocess isabelle_server failed to start.
  warnings.warn(f"Subprocess {self.name} failed to start.")
[2024-03-20 08:00:09,052] [Azure] [DEBUG] - azure
[2024-03-20 08:00:09,052] [Azure] [DEBUG] - azure
[2024-03-20 08:00:09,052] [Azure] [DEBUG] - azure
/workspace/huangyongfeng/ananke/example/pipeline/math_experiment/LEGO-Prover/lego_prover/env/process_monitor.py:80: UserWarning: Subprocess isabelle_server failed to start.
  warnings.warn(f"Subprocess {self.name} failed to start.")


Subprocess isabelle_server started with PID 2435797.
Subprocess isabelle_server started with PID 2435848.


/workspace/huangyongfeng/ananke/example/pipeline/math_experiment/LEGO-Prover/lego_prover/env/process_monitor.py:80: UserWarning: Subprocess isabelle_server failed to start.
  warnings.warn(f"Subprocess {self.name} failed to start.")


Subprocess isabelle_server started with PID 2435890.


...

/workspace/huangyongfeng/ananke/example/pipeline/math_experiment/LEGO-Prover/lego_prover/env/process_monitor.py:80: UserWarning: Subprocess isabelle_server failed to start.
  warnings.warn(f"Subprocess {self.name} failed to start.")


Subprocess isabelle_server started with PID 2447458.


/workspace/huangyongfeng/ananke/example/pipeline/math_experiment/LEGO-Prover/lego_prover/env/process_monitor.py:80: UserWarning: Subprocess isabelle_server failed to start.
  warnings.warn(f"Subprocess {self.name} failed to start.")


Subprocess isabelle_server started with PID 2448263.


/workspace/huangyongfeng/ananke/example/pipeline/math_experiment/LEGO-Prover/lego_prover/env/process_monitor.py:80: UserWarning: Subprocess isabelle_server failed to start.
  warnings.warn(f"Subprocess {self.name} failed to start.")
